In [1]:
import importlib
from pprint import pprint

import pandas as pd
from cuery import search
from cuery.seo import geo
from cuery.utils import set_env

set_env(apify_secrets=False)

# LLM prompting with web search

In [ ]:
# model = "openai/gpt-4.1-mini"
# model = "google/gemini-2.5-flash"
# model = "xai/grok-3"
# model = "deepseek/deepseek-chat"

prompts = [
    "Mejores SUVs en España 2025 precio-calidad",
    "Cuáles son los mejores SUVs chinos dispnibles en España 2025",
]

responses = await search.query(prompts, model=model, max_concurrent=10, use_search=True)
responses.to_pandas()

# Competitor extraction

In [ ]:
importlib.reload(geo)

competitors = await geo.find_competitors(
    brands=["BYD"],
    sector="automotive",
    market="Spain",
    max_count=5,
)
competitors

In [ ]:
competitors = ["toyota", "kia", "mg", "renault", "omoda"]

In [ ]:
from cuery.seo import geo

importlib.reload(geo)

prompts = await geo.commercial_prompts(
    n=10,
    language="en",
    sector=None,  # "automotive",
    market=None,  # "Spain",
    brands=["BYD"] + competitors,
    include_brands="never",
)

pprint(prompts)

In [ ]:
importlib.reload(search)
importlib.reload(geo)

use_search = False
models = ["openai/gpt-4.1", "google/gemini-2.5-flash"]
# models = ["deepseek/deepseek-chat"]

df = await geo.query_with_models(prompts, models, use_search=use_search)
df

In [ ]:
df.to_parquet("geo_results.parquet", index=False)

In [ ]:
df = df[[col for col in df if not any(x in col for x in ("_ranks", "_pos"))]].copy()
df

In [ ]:
importlib.reload(geo)

df2 = geo.add_brand_ranks(
    df,
    brands=["Opel", "Dacia", "BYD", "Kia", "Renault", "MG", "Omoda", "Jaecoo"],
)
df2

# All-in-one

In [3]:
importlib.reload(geo)

params = {
    "models": ["openai/gpt-4.1", "google/gemini-2.5-flash", "deepseek/deepseek-chat"],
    "prompts_seed": ["best electric SUVs for families in 2024"],
    "prompts_max": 10,
    "prompts_language": "English",
    "brands_seed": ["BYD"],
    "brands_max": 10,
    "brands_model": "openai/gpt-4.1",
    "brands_in_prompt": "never",
    "own_brands": ["Peugeot", "Opel"],
    "sector": "automotive",
    "market": "Spain",
    "use_search": True,
}

cfg = geo.GeoConfig(**params)
print(cfg)

result = await geo.analyse(cfg)
result

2025-09-11 17:31:32 [cuery|INFO] Querying LLMs with

GeoConfig
—————————
{
  "models": [
    "openai/gpt-4.1",
    "google/gemini-2.5-flash",
    "deepseek/deepseek-chat"
  ],
  "prompts_seed": [
    "best electric SUVs for families in 2024"
  ],
  "prompts_max": 10,
  "prompts_language": "English",
  "brands_seed": [
    "BYD"
  ],
  "brands_max": 10,
  "brands_model": "openai/gpt-4.1",
  "brands_in_prompt": "never",
  "own_brands": [
    "Peugeot",
    "Opel"
  ],
  "sector": "automotive",
  "market": "Spain",
  "use_search": true
}

2025-09-11 17:31:32 [cuery|INFO] Using these seed brands: ['Opel', 'Peugeot', 'BYD']
2025-09-11 17:31:32 [cuery|INFO] Searching for competitor brands with prompt:
Identify up to 10 real competing brands for ['Opel', 'Peugeot', 'BYD'] in the 'automotive' sector in the 'Spain' market


GeoConfig
—————————
{
  "models": [
    "openai/gpt-4.1",
    "google/gemini-2.5-flash",
    "deepseek/deepseek-chat"
  ],
  "prompts_seed": [
    "best electric SUVs for families in 2024"
  ],
  "prompts_max": 10,
  "prompts_language": "English",
  "brands_seed": [
    "BYD"
  ],
  "brands_max": 10,
  "brands_model": "openai/gpt-4.1",
  "brands_in_prompt": "never",
  "own_brands": [
    "Peugeot",
    "Opel"
  ],
  "sector": "automotive",
  "market": "Spain",
  "use_search": true
}



Gathering searches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-11 17:31:40 [cuery|INFO] Extracting competitor names from search result
2025-09-11 17:31:42 [cuery|INFO] Found these competitors brands: ['dacia', 'seat', 'hyundai', 'fiat', 'mercedes-benz', 'mg', 'ford', 'renault', 'volkswagen', 'kia', 'toyota']
2025-09-11 17:31:42 [cuery|INFO] Generating commercial search queries with prompt:
Generate 9 unique, concise search queries for consumer/commercial intent. Cover realistic user intents like comparisons, transactional queries, alternatives, trust/regulatory, and location nuances. Generate prompts in the 'English' language. Focus on the 'automotive' sector. Focus on the 'Spain' market. If brand context helps, consider these brands: ['Opel', 'Peugeot', 'BYD', 'dacia', 'seat', 'hyundai', 'fiat', 'mercedes-benz', 'mg', 'ford', 'renault', 'volkswagen', 'kia', 'toyota']. Do not mention any brand names explicitly in the queries though. Strictly return a JSON array of strings. No numbering, no prose, no code fences.
2025-09-11 17:31:45 [cuery|

Gathering searches:   0%|          | 0/30 [00:00<?, ?it/s]

2025-09-11 17:34:28 [cuery|INFO] Analysing brand ranks
2025-09-11 17:34:28 [cuery|INFO] Summarising own brand and competitor ranks
2025-09-11 17:34:28 [cuery|INFO] Got results dataframe:
                                             prompt  \
0          SUVs with lowest maintenance costs Spain   
1           best electric SUVs for families in 2024   
2      best electric cars for city driving in Spain   
3               car leasing vs buying options Spain   
4               compare hybrid vs diesel cars Spain   
5      eco-friendly car incentives government Spain   
6  family car alternatives under 20,000 euros Spain   
7               latest car safety regulations Spain   
8            trusted car dealerships near Barcelona   
9                 used cars warranty policies Spain   

                         text_deepseek/deepseek-chat  \
0  Based on my analysis of SUV maintenance costs ...   
1  Based on my analysis of the 2024 electric SUV ...   
2  Based on my analysis of the electric

prompt  \
0          SUVs with lowest maintenance costs Spain   
1           best electric SUVs for families in 2024   
2      best electric cars for city driving in Spain   
3               car leasing vs buying options Spain   
4               compare hybrid vs diesel cars Spain   
5      eco-friendly car incentives government Spain   
6  family car alternatives under 20,000 euros Spain   
7               latest car safety regulations Spain   
8            trusted car dealerships near Barcelona   
9                 used cars warranty policies Spain   

                         text_deepseek/deepseek-chat  \
0  Based on my analysis of SUV maintenance costs ...   
1  Based on my analysis of the 2024 electric SUV ...   
2  Based on my analysis of the electric vehicle m...   
3  # Car Leasing vs Buying Options in Spain\n\n##...   
4  ## Comparison of Hybrid vs Diesel Cars in Spai...   
5  Based on my search, here are the current eco-f...   
6  Based on my research, here are some excellent ...   
7  I understand you're looking for information ab...   
8  I understand you're looking for trusted car de...   
9  I understand you're looking for information ab...   

                        text_google/gemini-2.5-flash  \
0  When considering SUVs with the lowest maintena...   
1  When considering the best electric SUVs for fa...   
2  For urban dwellers in Spain seeking efficient ...   
3  ## Car Leasing vs. Buying in Spain: A Comprehe...   
4  In Spain, the automotive landscape is undergoi...   
5  The Spanish government offers a range of incen...   
6  For families in Spain looking for car alternat...   
7  Spain is implementing several new car safety r...   
8  When looking for a trusted car dealership near...   
9  In Spain, warranty policies for used cars vary...   

                                 text_openai/gpt-4.1  \
0  Here are some of the SUVs that offer the **low...   
1  Here’s a refined look at the **best electric S...   
2  Aquí tienes un análisis actualizado (septiembr...   
3  Aquí tienes un análisis detallado y actualizad...   
4  Here’s a comprehensive comparison of hybrid ve...   
5  Here’s a detailed and up-to-date overview of t...   
6  Here are some great **family car options under...   
7  Here are the **latest car safety regulations i...   
8  If you're looking for trusted car dealerships ...   
9  Here’s a detailed overview of used‑car warrant...   

  references_deepseek/deepseek-chat  \
0                                []   
1                                []   
2                                []   
3                                []   
4                                []   
5                                []   
6                                []   
7                                []   
8                                []   
9                                []   

                  references_google/gemini-2.5-flash  \
0  [{'title': 'caranddriver.com', 'url': 'https:/...   
1  [{'title': 'kbb.com', 'url': 'https://www.kbb....   
2  [{'title': 'alegria-realestate.com', 'url': 'h...   
3  [{'title': 'blancacarsleasing.com', 'url': 'ht...   
4  [{'title': 'autovistagroup.com', 'url': 'https...   
5  [{'title': 'europa.eu', 'url': 'https://altern...   
6  [{'title': 'thelocal.es', 'url': 'https://www....   
7  [{'title': 'euroweeklynews.com', 'url': 'https...   
8  [{'title': 'youtube.com', 'url': 'https://www....   
9  [{'title': 'murciatoday.com', 'url': 'https://...   

                           references_openai/gpt-4.1  \
0  [{'title': '10 Best SUVs With The Lowest Maint...   
1  [{'title': '12 Best Electric SUVs of 2024 | Ke...   
2  [{'title': 'Hyundai Inster, Coche Urbano del A...   
3  [{'title': '¿Comprar o alquilar? ¿Cuál es la o...   
4  [{'title': 'Uplifting BEV result accelerates n...   
5  [{'title': 'Spain extends Moves III programme ...   
6  [{'title': 'The 12 Most Popular New Cars Under...   
7  [{'title': 'Spain to mandate V-16 emergency be...   
8  [{'title': 'Tu red de concesionari

In [10]:
all_models = []
for prov, models in search.VALID_MODELS.items():
    for model in models:
        all_models.append(f"{prov}/{model}")

all_models

['openai/gpt-4o-mini',
 'openai/gpt-4o',
 'openai/gpt-4.1-mini',
 'openai/gpt-4.1',
 'openai/o4-mini',
 'openai/o3',
 'openai/gpt-5 with reasoning levels low, medium and high',
 'google/gemini-2.5-pro',
 'google/gemini-2.5-flash',
 'google/gemini-2.5-flash-lite',
 'google/gemini-2.0-flash',
 'xai/grok-4',
 'xai/grok-3',
 'xai/grok-3-mini',
 'deepseek/deepseek-chat',
 'deepseek/deepseek-reasoner']

In [3]:
result.to_parquet("geo_results.parquet", index=False)

In [46]:
result = pd.read_parquet("geo_results.parquet")
result

prompt  \
0               alternatives to diesel cars in Spain   
1            best electric SUVs for families in 2024   
2             best hybrid SUVs for families in Spain   
3  cheapest automatic sedans with high safety rat...   
4  compare compact crossover models by fuel effic...   
5  dealerships with top-rated after-sales service...   
6  eco-friendly cars eligible for Spanish governm...   
7          electric car financing options 2024 Spain   
8       regulations for importing used cars to Spain   
9    reliable small cars for first-time buyers Spain   

                         text_deepseek/deepseek-chat  \
0  Here are some alternatives to diesel cars in S...   
1  Based on my analysis of the electric SUV marke...   
2  Based on my analysis of the Spanish automotive...   
3  Based on my research, here are the cheapest au...   
4  I understand you're looking to compare compact...   
5  I currently don't have access to real-time dea...   
6  Based on the information available to me, I ca...   
7  # Electric Car Financing Options in Spain 2024...   
8  To provide you with accurate and up-to-date in...   
9  ## Reliable Small Cars for First-Time Buyers i...   

                        text_google/gemini-2.5-flash  \
0  Spain offers a growing number of alternatives ...   
1  For families seeking an eco-friendly and pract...   
2  When considering the best hybrid SUVs for fami...   
3  To identify the cheapest automatic sedans with...   
4  When comparing compact crossover models by fue...   
5  When seeking dealerships with top-rated after-...   
6  The Spanish government has extended its MOVES ...   
7  In Spain, electric car financing in 2024 is si...   
8  Importing a used car to Spain involves a multi...   
9  For first-time car buyers in Spain seeking rel...   

                                 text_openai/gpt-4.1  \
0  Here’s a comprehensive look at the **alternati...   
1  Here’s a comprehensive guide to the best elect...   
2  Here’s a thoroughly researched guide to the be...   
3  Aquí tienes algunas de las opciones más asequi...   
4  Here’s a comparison of compact crossover (comp...   
5  Aquí tienes una selección de concesionarios de...   
6  Here’s the latest information (as of September...   
7  Here’s a comprehensive overview of electric‑ca...   
8  Here’s a **comprehensive guide** (as of mid‑20...   
9  If you're looking for reliable small cars in S...   

  references_deepseek/deepseek-chat  \
0                                []   
1                                []   
2                                []   
3                                []   
4                                []   
5                                []   
6                                []   
7                                []   
8                                []   
9                                []   

                  references_google/gemini-2.5-flash  \
0  [{'title': 'europa.eu', 'url': 'https://altern...   
1  [{'title': 'kbb.com', 'url': 'https://www.kbb....   
2  [{'title': 'reddit.com', 'url': 'https://www.r...   
3  [{'title': 'ccarprice.com', 'url': 'https://ww...   
4                                                 []   
5  [{'title': 'canalcar.es', 'url': 'https://www....   
6  [{'title': 'lamoncloa.gob.es', 'url': 'https:/...   
7  [{'title': 'powy.energy', 'url': 'https://powy...   
8  [{'title': 'immofy.eu', 'url': 'https://blog.i...   
9  [{'title': 'autobahn.es', 'url': 'https://auto...   

                           references_openai/gpt-4.1  \
0  [{'title': 'Spain's EV Revolution: Chinese Bra...   
1  [{'title': 'What Makes the Best Electric SUVs ...   
2  [{'title': 'Los mejores SUV híbridos enchufabl...   
3  [{'title': 'Los coches más baratos en 2024, se...   
4  [{'title': 'Toyota RAV4', 'url': 'https://it.w...   
5  [{'title': 'Los Mejores Concesionarios de Madr...   
6  [{'title': 'La Moncloa. 01/04/2025. The Govern...   
7  [{'title': 'Spain brings back €10,000 electric...   
8  [{'title': 'Tax Agency: 

In [47]:
importlib.reload(geo)

own = ["Peugeot"]
competitors = [
    "citroën",
    "mercedes-benz",
    "volkswagen",
    "mg",
    "dacia",
    "renault",
    "toyota",
    "hyundai",
    "seat",
    "kia",
]

geo.summarize_ranks(result, models=cfg.models, own=own, competitors=competitors, emoji_flags=True)

own_in_txt_openai/gpt-4.1
cmp_in_txt_openai/gpt-4.1
own_in_refs_openai/gpt-4.1
cmp_in_refs_openai/gpt-4.1
own_in_txt_google/gemini-2.5-flash
cmp_in_txt_google/gemini-2.5-flash
own_in_refs_google/gemini-2.5-flash
cmp_in_refs_google/gemini-2.5-flash
own_in_txt_deepseek/deepseek-chat
cmp_in_txt_deepseek/deepseek-chat
own_in_refs_deepseek/deepseek-chat
cmp_in_refs_deepseek/deepseek-chat


prompt  \
0               alternatives to diesel cars in Spain   
1            best electric SUVs for families in 2024   
2             best hybrid SUVs for families in Spain   
3  cheapest automatic sedans with high safety rat...   
4  compare compact crossover models by fuel effic...   
5  dealerships with top-rated after-sales service...   
6  eco-friendly cars eligible for Spanish governm...   
7          electric car financing options 2024 Spain   
8       regulations for importing used cars to Spain   
9    reliable small cars for first-time buyers Spain   

                         text_deepseek/deepseek-chat  \
0  Here are some alternatives to diesel cars in S...   
1  Based on my analysis of the electric SUV marke...   
2  Based on my analysis of the Spanish automotive...   
3  Based on my research, here are the cheapest au...   
4  I understand you're looking to compare compact...   
5  I currently don't have access to real-time dea...   
6  Based on the information available to me, I ca...   
7  # Electric Car Financing Options in Spain 2024...   
8  To provide you with accurate and up-to-date in...   
9  ## Reliable Small Cars for First-Time Buyers i...   

                        text_google/gemini-2.5-flash  \
0  Spain offers a growing number of alternatives ...   
1  For families seeking an eco-friendly and pract...   
2  When considering the best hybrid SUVs for fami...   
3  To identify the cheapest automatic sedans with...   
4  When comparing compact crossover models by fue...   
5  When seeking dealerships with top-rated after-...   
6  The Spanish government has extended its MOVES ...   
7  In Spain, electric car financing in 2024 is si...   
8  Importing a used car to Spain involves a multi...   
9  For first-time car buyers in Spain seeking rel...   

                                 text_openai/gpt-4.1  \
0  Here’s a comprehensive look at the **alternati...   
1  Here’s a comprehensive guide to the best elect...   
2  Here’s a thoroughly researched guide to the be...   
3  Aquí tienes algunas de las opciones más asequi...   
4  Here’s a comparison of compact crossover (comp...   
5  Aquí tienes una selección de concesionarios de...   
6  Here’s the latest information (as of September...   
7  Here’s a comprehensive overview of electric‑ca...   
8  Here’s a **comprehensive guide** (as of mid‑20...   
9  If you're looking for reliable small cars in S...   

  references_deepseek/deepseek-chat  \
0                                []   
1                                []   
2                                []   
3                                []   
4                                []   
5                                []   
6                                []   
7                                []   
8                                []   
9                                []   

                  references_google/gemini-2.5-flash  \
0  [{'title': 'europa.eu', 'url': 'https://altern...   
1  [{'title': 'kbb.com', 'url': 'https://www.kbb....   
2  [{'title': 'reddit.com', 'url': 'https://www.r...   
3  [{'title': 'ccarprice.com', 'url': 'https://ww...   
4                                                 []   
5  [{'title': 'canalcar.es', 'url': 'https://www....   
6  [{'title': 'lamoncloa.gob.es', 'url': 'https:/...   
7  [{'title': 'powy.energy', 'url': 'https://powy...   
8  [{'title': 'immofy.eu', 'url': 'https://blog.i...   
9  [{'title': 'autobahn.es', 'url': 'https://auto...   

                           references_openai/gpt-4.1  \
0  [{'title': 'Spain's EV Revolution: Chinese Bra...   
1  [{'title': 'What Makes the Best Electric SUVs ...   
2  [{'title': 'Los mejores SUV híbridos enchufabl...   
3  [{'title': 'Los coches más baratos en 2024, se...   
4  [{'title': 'Toyota RAV4', 'url': 'https://it.w...   
5  [{'title': 'Los Mejores Concesionarios de Madr...   
6  [{'title': 'La Moncloa. 01/04/2025. The Govern...   
7  [{'title': 'Spain brings back €10,000 electric...   
8  [{'title': 'Tax Agency: 

In [ ]:
l = [1, 2, 3]
c = l
c.append(4)
l

[1, 2, 3, 4]

In [41]:
importlib.reload(geo)

result = result[[col for col in result if not any(x in col for x in ("_ranks", "_pos"))]].copy()
result = geo.add_brand_ranks(
    result,
    brands=cfg.brands_seed
    + [
        "citroën",
        "mercedes-benz",
        "volkswagen",
        "mg",
        "dacia",
        "renault",
        "toyota",
        "hyundai",
        "seat",
        "kia",
    ],
)
result

prompt  \
0        approved used cars warranty Spain   
1  best electric SUVs for families in 2024   
2              best suv offers Madrid 2024   
3                 car safety ratings Spain   
4          compact crossover reviews Spain   
5      compare hybrid vs diesel cars Spain   
6   eco-friendly city cars available Spain   
7        electric car tax incentives Spain   
8      family hatchback alternatives Spain   
9           lease vs finance new car Spain   

                         text_deepseek/deepseek-chat  \
0  I understand you're looking for information ab...   
1  Based on my research, here are the best electr...   
2  I understand you're looking for the best SUV o...   
3  I understand you're looking for car safety rat...   
4  I understand you're looking for compact crosso...   
5  ## Hybrid vs Diesel Cars in Spain: Comparison\...   
6  Based on your query about eco-friendly city ca...   
7  ## Electric Car Tax Incentives in Spain\n\n###...   
8  Based on your query about family hatchback alt...   
9  When deciding between leasing and financing a ...   

                        text_google/gemini-2.5-flash  \
0  In Spain, the warranty for approved used cars ...   
1  For families seeking an eco-friendly and pract...   
2  While specific "best" offers can be subjective...   
3  Car safety ratings in Spain are primarily dete...   
4  In the Spanish market, compact crossovers cont...   
5  ## Hybrid vs. Diesel Cars in Spain: A Comprehe...   
6  Spain offers a growing selection of eco-friend...   
7  Spain offers several tax incentives and subsid...   
8  For families in Spain looking for alternatives...   
9  When considering a new car in Spain, both leas...   

                                 text_openai/gpt-4.1  \
0  If you're looking into warranties for "approve...   
1  Here’s a well-rounded overview of the best ele...   
2  Aquí tienes algunas de las **mejores ofertas d...   
3  Aquí tienes una visión clara y actualizada sob...   
4  Aquí tienes una visión actualizada (septiembre...   
5  Here’s a thorough comparison of hybrid versus ...   
6  Here’s a comprehensive overview of eco‑friendl...   
7  Here is an up-to-date overview (as of Septembe...   
8  If you're looking for practical alternatives t...   
9  Aquí tienes una comparativa clara y detallada ...   

  references_deepseek/deepseek-chat  \
0                                []   
1                                []   
2                                []   
3                                []   
4                                []   
5                                []   
6                                []   
7                                []   
8                                []   
9                                []   

                  references_google/gemini-2.5-flash  \
0  [{'title': 'mylawyerinspain.com', 'url': 'http...   
1  [{'title': 'kbb.com', 'url': 'https://www.kbb....   
2  [{'title': 'coches.net', 'url': 'https://www.c...   
3  [{'title': 'byd.com', 'url': 'https://www.byd....   
4  [{'title': 'youtube.com', 'url': 'https://www....   
5  [{'title': 'globalpetrolprices.com', 'url': 'h...   
6  [{'title': 'euroweeklynews.com', 'url': 'https...   
7  [{'title': 'electrive.com', 'url': 'https://ww...   
8  [{'title': '6wresearch.com', 'url': 'https://v...   
9  [{'title': 'autoleaserenting.es', 'url': 'http...   

                           references_openai/gpt-4.1  \
0  [{'title': '! Spanish News Today - Extended Wa...   
1  [{'title': 'Edmunds Top Rated Awards for 2024 ...   
2  [{'title': 'BYD llega a Grupo Gamboa: descubre...   
3  [{'title': 'DGT - Clasificación de seguridad d...   
4  [{'title': 'Citroën C3 Aircross, Coche del Año...   
5  [{'title': 'Car sales in Spain by energy sourc...   
6  [{'title': 'Best Small Electric Car 2025: Pric...   
7  [{'title': 'Spain extends Moves III programme ...   
8  [{'title': 'Top 5 Station Wagons to Buy in 202...   
9  [{'title': 'Leasing vs Renting de Coches en Es...   

              